In [1]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
import statsmodels.api as sm
import re

In [2]:
data_dir = "C:/Users/marik/Downloads/raw_data/raw_extracted"
file_list = os.listdir(data_dir)
raw_data ={os.path.splitext(filename)[0]: pd.read_excel(os.path.join(data_dir, filename), header=0) 
        for filename in file_list}

raw_data = {key[4:]: value for key, value in raw_data.items()} # remove 'TPP_' from the keys

In [3]:
df_test = raw_data[list(raw_data.keys())[0]]
df_test.head()

,Protein_ID,norm_FC_TMT126_P023755_B_subtilis,norm_FC_TMT127L_P023755_B_subtilis,norm_FC_TMT127H_P023755_B_subtilis,norm_FC_TMT128L_P023755_B_subtilis,norm_FC_TMT128H_P023755_B_subtilis,norm_FC_TMT129L_P023755_B_subtilis,norm_FC_TMT129H_P023755_B_subtilis,norm_FC_TMT130L_P023755_B_subtilis,norm_FC_TMT130H_P023755_B_subtilis,...,plot,protein_identified_in_P023755_B_subtilis,model_converged_P023755_B_subtilis,sufficient_data_for_fit_P023755_B_subtilis,Proteinname_P023755_B_subtilis,numSpec_P023755_B_subtilis,gene_name,uniprot_ac,auc,plot_link
0,C0H3Q1_ytzI,0.999203,1.409274,1.821193,1.223887,1.208013,1.023653,0.976470,1.045403,0.908158,...,Melting_Curves/meltCurve_C0H3Q1_ytzI.pdf,Yes,Yes,Yes,Uncharacterized membrane protein YtzI,1,ytzI,C0H3Q1,0.977245,Melting_Curves/meltCurve_C0H3Q1_ytzI.pdf
1,C0H3V2_mtlF,0.999203,1.225128,1.102669,1.317735,0.896313,0.865444,0.728182,0.705247,0.402074,...,Melting_Curves/meltCurve_C0H3V2_mtlF.pdf,Yes,Yes,Yes,Mannitol-specific phosphotransferase enzyme II...,1,mtlF,C0H3V2,0.831531,Melting_Curves/meltCurve_C0H3V2_mtlF.pdf
2,C0H3V8_yyzM,0.999203,0.971943,1.028080,0.673125,0.500140,0.396486,0.282503,0.224742,0.210854,...,Melting_Curves/meltCurve_C0H3V8_yyzM.pdf,Yes,Yes,Yes,Uncharacterized protein YyzM,2,yyzM,C0H3V8,0.530496,Melting_Curves/meltCurve_C0H3V8_yyzM.pdf
3,C0H3Y1_yhzD,0.999203,0.466371,0.639208,0.578452,0.461037,0.661968,0.350363,0.085139,0.057781,...,Melting_Curves/meltCurve_C0H3Y1_yhzD.pdf,Yes,Yes,Yes,Uncharacterized protein YhzD,1,yhzD,C0H3Y1,0.434036,Melting_Curves/meltCurve_C0H3Y1_yhzD.pdf
4,C0H3Z2_yjzH,0.999203,1.351992,1.044422,1.093136,0.929048,0.821314,0.664804,0.635900,0.698621,...,Melting_Curves/meltCurve_C0H3Z2_yjzH.pdf,Yes,Yes,Yes,Uncharacterized protein YjzH,3,yjzH,C0H3Z2,0.841153,Melting_Curves/meltCurve_C0H3Z2_yjzH.pdf


In [4]:
col_name = pd.read_csv('C:/Users/marik/Downloads/raw_data/column_header/column_header.csv')
#col_drop = col_name.drop(axis = 1, labels =['Protein_ID', 'meltPoint_P023428_E.coli', 'Proteinname_P023428_E.coli', 'gene_name', 'uniprot_ac', 'auc'])
col_list = col_name.columns.tolist()
drop_list = ['norm_FC_TMT126_P023428_E.coli', 'norm_FC_TMT127L_P023428_E.coli', 'norm_FC_TMT127H_P023428_E.coli', 
             'norm_FC_TMT128L_P023428_E.coli', 'norm_FC_TMT128H_P023428_E.coli', 'norm_FC_TMT129L_P023428_E.coli', 'norm_FC_TMT129H_P023428_E.coli', 
             'norm_FC_TMT130L_P023428_E.coli', 'norm_FC_TMT130H_P023428_E.coli', 'norm_FC_TMT131L_P023428_E.coli', 'a_P023428_E.coli', 
             'b_P023428_E.coli', 'inflPoint_P023428_E.coli', 'slope_P023428_E.coli', 'plateau_P023428_E.coli', 
             'R_sq_P023428_E.coli', 'plot', 'protein_identified_in_P023428_E.coli', 'model_converged_P023428_E.coli', 
             'sufficient_data_for_fit_P023428_E.coli', 'numSpec_P023428_E.coli', 'plot_link']
key_list = list(raw_data.keys())

# Rename columns based on col_list (equal to drop_list, except for elements not included)
for n in range(len(key_list)):
    old_cols = list(raw_data[key_list[n]].columns)
    col_list = col_name[:len(old_cols)]  # Ensure col_name has enough elements
    rename_dict = dict(zip(old_cols, col_list))
    raw_data[key_list[n]] = raw_data[key_list[n]].rename(columns=rename_dict)

 # Drop columns in drop_list
for n in range(len(key_list)):
    raw_data[key_list[n]] = raw_data[key_list[n]].drop(columns = drop_list, axis=1, errors = 'ignore')

# Rename columns to be consistent across all dataframes
# somhow not working, but not really important
""" raw_data[key_list[0]].head()
for n in range(len(key_list)):
    raw_data[key_list[n]] = raw_data[key_list[n]].rename(columns={
        'meltPoint__P023428_E.coli': 'meltPoint', 'Proteinname_P023428_E.coli': 'Proteinname'}) """

" raw_data[key_list[0]].head()\nfor n in range(len(key_list)):\n    raw_data[key_list[n]] = raw_data[key_list[n]].rename(columns={\n        'meltPoint__P023428_E.coli': 'meltPoint', 'Proteinname_P023428_E.coli': 'Proteinname'}) "

In [37]:
df_test = raw_data[list(raw_data.keys())[0]]
df_test.head()


,Protein_ID,meltPoint_P023428_E.coli,Proteinname_P023428_E.coli,gene_name,uniprot_ac,auc
0,C0H3Q1_ytzI,NaN,Uncharacterized membrane protein YtzI,ytzI,C0H3Q1,0.977245
1,C0H3V2_mtlF,56.693844,Mannitol-specific phosphotransferase enzyme II...,mtlF,C0H3V2,0.831531
2,C0H3V8_yyzM,43.373369,Uncharacterized protein YyzM,yyzM,C0H3V8,0.530496
3,C0H3Y1_yhzD,40.693719,Uncharacterized protein YhzD,yhzD,C0H3Y1,0.434036
4,C0H3Z2_yjzH,NaN,Uncharacterized protein YjzH,yjzH,C0H3Z2,0.841153


In [5]:
data = {}
for i in range(len(raw_data.keys())):
    df = raw_data[list(raw_data.keys())[i]].copy()
    if df.columns[0] != 'Sample':
        df.insert(loc=0, column = 'Sample', 
                  value = pd.Series(str(list(raw_data.keys())[i]), index = df.index))
        data[list(raw_data.keys())[i]] = df
    else:
        data[list(raw_data.keys())[i]] = df

In [8]:
data_prokaryotes = pd.concat(data.values(), axis = 0, ignore_index=True, join='inner')
data_prokaryotes.head()

,Sample,Protein_ID,meltPoint_P023428_E.coli,Proteinname_P023428_E.coli,gene_name,uniprot_ac,auc
0,B.subtilis_P023755,C0H3Q1_ytzI,NaN,Uncharacterized membrane protein YtzI,ytzI,C0H3Q1,0.977245
1,B.subtilis_P023755,C0H3V2_mtlF,56.693844,Mannitol-specific phosphotransferase enzyme II...,mtlF,C0H3V2,0.831531
2,B.subtilis_P023755,C0H3V8_yyzM,43.373369,Uncharacterized protein YyzM,yyzM,C0H3V8,0.530496
3,B.subtilis_P023755,C0H3Y1_yhzD,40.693719,Uncharacterized protein YhzD,yhzD,C0H3Y1,0.434036
4,B.subtilis_P023755,C0H3Z2_yjzH,NaN,Uncharacterized protein YjzH,yjzH,C0H3Z2,0.841153
